In [1]:
!pip install kss

In [2]:
import pickle

# load
with open('id.pickle', 'rb') as f:
    id_news = pickle.load(f)

In [3]:
with open('contents.pickle', 'rb') as f:
    news = pickle.load(f)

In [4]:
len(news)
del news[0]
del news[1]
del news[2]
del news[3]
del news[4]
del news[5]
len(news)

532

In [5]:
del id_news[0]
del id_news[1]
del id_news[2]
del id_news[3]
del id_news[4]
del id_news[5]

In [6]:
len(id_news)

532

In [7]:
import platform
from collections import Counter

import numpy as np

# tokenizer import
from konlpy.tag import Okt, Komoran, Hannanum, Kkma

#운영체제에 따라 mecab설치 방법이 다름.
if platform.system() == "Windows":
    try:
        from eunjeon import Mecab
    except:
        print("please install eunjeon module")
else:  # Ubuntu일 경우
    from konlpy.tag import Mecab

from typing import List, Callable, Union, Any, TypeVar, Tuple, Dict

from functools import partial
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
import gensim
from gensim import corpora
import kss

from gensim.models import LdaModel


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [8]:
import warnings
warnings.filterwarnings(action='ignore')


In [9]:
import gensim
from gensim import corpora
import kss
import time
from konlpy.tag import Mecab
import random

In [10]:
print(len(news))
lines = ""

for i in news:
    lines = lines+i


532


In [11]:
#불용어 사전에 "명" 추가
stop = "말,건,분,회,간,내,수,거,게,명,직전,아,휴,아이구,아이쿠,아이고,어,나,우리,저희,따라,의해,을,를,에,의,가,으로,로,에게,뿐이다,의거하여,근거하여,입각하여,기준으로,예하면,예를 들면,예를 들자면,저,소인,소생,저희,지말고,하지마,하지마라,다른,물론,또한,그리고,비길수 없다,해서는 안된다,뿐만 아니라,만이 아니다,만은 아니다,막론하고,관계없이,그치지 않다,그러나,그런데,하지만,든간에,논하지 않다,따지지 않다,설사,비록,더라도,아니면,만 못하다,하는 편이 낫다,불문하고,향하여,향해서,향하다,쪽으로,틈타,이용하여,타다,오르다,제외하고,이 외에,이 밖에,하여야,비로소,한다면 몰라도,외에도,이곳,여기,부터,기점으로,따라서,할 생각이다,하려고하다,이리하여,그리하여,그렇게 함으로써,하지만,일때,할때,앞에서,중에서,보는데서,으로써,로써,까지,해야한다,일것이다,반드시,할줄알다,할수있다,할수있어,임에 틀림없다,한다면,등,등등,제,겨우,단지,다만,할뿐,딩동,댕그,대해서,대하여,대하면,훨씬,얼마나,얼마만큼,얼마큼,남짓,여,얼마간,약간,다소,좀,조금,다수,몇,얼마,지만,하물며,또한,그러나,그렇지만,하지만,이외에도,대해 말하자면,뿐이다,다음에,반대로,반대로 말하자면,이와 반대로,바꾸어서 말하면,바꾸어서 한다면,만약,그렇지않으면,까악,툭,딱,삐걱거리다,보드득,비걱거리다,꽈당,응당,해야한다,에 가서,각,각각,여러분,각종,각자,제각기,하도록하다,와,과,그러므로,그래서,고로,한 까닭에,하기 때문에,거니와,이지만,대하여,관하여,관한,과연,실로,아니나다를가,생각한대로,진짜로,한적이있다,하곤하였다,하,하하,허허,아하,거바,와,오,왜,어째서,무엇때문에,어찌,하겠는가,무슨,어디,어느곳,더군다나,하물며,더욱이는,어느때,언제,야,이봐,어이,여보시오,흐흐,흥,휴,헉헉,헐떡헐떡,영차,여차,어기여차,끙끙,아야,앗,아야,콸콸,졸졸,좍좍,뚝뚝,주룩주룩,솨,우르르,그래도,또,그리고,바꾸어말하면,바꾸어말하자면,혹은,혹시,답다,및,그에 따르는,때가 되어,즉,지든지,설령,가령,하더라도,할지라도,일지라도,지든지,몇,거의,하마터면,인젠,이젠,된바에야,된이상,만큼,어찌됏든,그위에,게다가,점에서 보아,비추어 보아,고려하면,하게될것이다,일것이다,비교적,좀,보다더,비하면,시키다,하게하다,할만하다,의해서,연이서,이어서,잇따라,뒤따라,뒤이어,결국,의지하여,기대여,통하여,자마자,더욱더,불구하고,얼마든지,마음대로,주저하지 않고,곧,즉시,바로,당장,하자마자,밖에 안된다,하면된다,그래,그렇지,요컨대,다시 말하자면,바꿔 말하면,즉,구체적으로,말하자면,시작하여,시초에,이상,허,헉,허걱,바와같이,해도좋다,해도된다,게다가,더구나,하물며,와르르,팍,퍽,펄렁,동안,이래,하고있었다,이었다,에서,로부터,까지,예하면,했어요,해요,함께,같이,더불어,마저,마저도,양자,모두,습니다,가까스로,하려고하다,즈음하여,다른,다른 방면으로,해봐요,습니까,했어요,말할것도 없고,무릎쓰고,개의치않고,하는것만 못하다,하는것이 낫다,매,매번,들,모,어느것,어느,로써,갖고말하자면,어디,어느쪽,어느것,어느해,어느 년도,라 해도,언젠가,어떤것,어느것,저기,저쪽,저것,그때,그럼,그러면,요만한걸,그래,그때,저것만큼,그저,이르기까지,할 줄 안다,할 힘이 있다,너,너희,당신,어찌,설마,차라리,할지언정,할지라도,할망정,할지언정,구토하다,게우다,토하다,메쓰겁다,옆사람,퉤,쳇,의거하여,근거하여,의해,따라,힘입어,그,다음,버금,두번째로,기타,첫번째로,나머지는,그중에서,견지에서,형식으로 쓰여,입장에서,위해서단지,의해되다,하도록시키다,뿐만아니라,반대로,전후,전자,앞의것,잠시,잠깐,하면서,그렇지만,다음에,그러한즉,그런즉,남들,아무거나,어찌하든지,같다,비슷하다,예컨대,이럴정도로,어떻게,만약,만일,위에서 서술한바와같이,인 듯하다,하지 않는다면,만약에,무엇,무슨,어느,어떤,아래윗,조차,한데,그럼에도 불구하고,여전히,심지어,까지도,조차도,하지 않도록,않기 위하여,때,시각,무렵,시간,동안,어때,어떠한,하여금,네,예,우선,누구,누가 알겠는가,아무도,줄은모른다,줄은 몰랏다,하는 김에,겸사겸사,하는바,그런 까닭에,한 이유는,그러니,그러니까,때문에,그,너희,그들,너희들,타인,것,것들,너,위하여,공동으로,동시에,하기 위하여,어찌하여,무엇때문에,붕붕,윙윙,나,우리,엉엉,휘익,윙윙,오호,아하,어쨋든,만 못하다,하기보다는,차라리,하는 편이 낫다,흐흐,놀라다,상대적으로 말하자면,마치,아니라면,쉿,그렇지 않으면,그렇지 않다면,안 그러면,아니었다면,하든지,아니면,이라면,좋아,알았어,하는것도,그만이다,어쩔수 없다,하나,일,일반적으로,일단,한켠으로는,오자마자,이렇게되면,이와같다면,전부,한마디,한항목,근거로,하기에,아울러,하지 않도록,않기 위해서,이르기까지,이 되다,로 인하여,까닭으로,이유만으로,이로 인하여,그래서,이 때문에,그러므로,그런 까닭에,알 수 있다,결론을 낼 수 있다,으로 인하여,있다,어떤것,관계가 있다,관련이 있다,연관되다,어떤것들,에 대해,이리하여,그리하여,여부,하기보다는,하느니,하면 할수록,운운,이러이러하다,하구나,하도다,다시말하면,다음으로,에 있다,에 달려 있다,우리,우리들,오히려,하기는한데,어떻게,어떻해,어찌됏어,어때,어째서,본대로,자,이,이쪽,여기,이것,이번,이렇게말하자면,이런,이러한,이와 같은,요만큼,요만한 것,얼마 안 되는 것,이만큼,이 정도의,이렇게 많은 것,이와 같다,이때,이렇구나,것과 같이,끼익,삐걱,따위,와 같은 사람들,부류의 사람들,왜냐하면,중의하나,오직,오로지,에 한하다,하기만 하면,도착하다,까지 미치다,도달하다,정도에 이르다,할 지경이다,결과에 이르다,관해서는,여러분,하고 있다,한 후,혼자,자기,자기집,자신,우에 종합한것과같이,총적으로 보면,총적으로 말하면,총적으로,대로 하다,으로서,참,그만이다,할 따름이다,쿵,탕탕,쾅쾅,둥둥,봐,봐라,아이야,아니,와아,응,아이,참나,년,월,일,령,영,일,이,삼,사,오,육,륙,칠,팔,구,이천육,이천칠,이천팔,이천구,하나,둘,셋,넷,다섯,여섯,일곱,여덟,아홉,령,영 "

In [12]:
news_doc = lines

stime = time.time()
text_list = kss.split_sentences(news_doc)
print(time.time()-stime)
k = []


3.552284002304077


In [13]:
stop_words = stop.split(',')
mecab = Mecab()
stime = time.time()
for i in news :
    sentences = mecab.nouns(i)
    result2 = []
    for word in sentences :
        if word not in stop_words:
            result2.append(word)
    k.append(result2)
print(time.time()-stime)
print(len(k))

2.5707802772521973
532


In [14]:
bigram = gensim.models.Phrases(k, min_count = 5, threshold = 0.000000000000000000001)

bigram_model = gensim.models.phrases.Phraser(bigram)

bigram_document = [bigram_model[nouns] for nouns in k]

for i in range(len(bigram_document)):
    if word in stop_words:
        bigram_document.drop(i)



In [15]:
id2word = corpora.Dictionary(bigram_document)
corpus = [id2word.doc2bow(doc) for doc in bigram_document]

In [16]:
from gensim.models import CoherenceModel

coherence_score=[]
t_min = 2
t_max = 20
for i in range(t_min,t_max):
    model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=i)
    coherence_model = CoherenceModel(model, texts=bigram_document, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model.get_coherence()
    print('n=',i,'\nCoherence Score: ', coherence_lda)
    coherence_score.append(coherence_lda)

n= 2 
Coherence Score:  0.32868904538971866
n= 3 
Coherence Score:  0.34301341187657314
n= 4 
Coherence Score:  0.35679825328563314
n= 5 
Coherence Score:  0.3069160133897018
n= 6 
Coherence Score:  0.3558779922976203
n= 7 
Coherence Score:  0.3183588447374054
n= 8 
Coherence Score:  0.317071624867696
n= 9 
Coherence Score:  0.33137558076547974
n= 10 
Coherence Score:  0.33626841700074095
n= 11 
Coherence Score:  0.32796817979325915
n= 12 
Coherence Score:  0.3097755602381748
n= 13 
Coherence Score:  0.32732058961415567
n= 14 
Coherence Score:  0.3574096441662385
n= 15 
Coherence Score:  0.34967056575938177
n= 16 
Coherence Score:  0.34250781760432125
n= 17 
Coherence Score:  0.36646201408094287
n= 18 
Coherence Score:  0.3324307822289059
n= 19 
Coherence Score:  0.3464056283997443


In [17]:
co_sc = np.array(coherence_score)
NUM_TOPICS = np.argmax(co_sc)+t_min

In [18]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=id2word, passes=15)
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.021*"문_대통령" + 0.013*"초선_의원" + 0.012*"의원" + 0.011*"대통령" + 0.009*"부회장_사면"')
(1, '0.009*"사령관" + 0.008*"미국" + 0.008*"장관" + 0.007*"북한" + 0.007*"인도_태평양"')
(2, '0.014*"의원" + 0.010*"차_의혹" + 0.008*"유족_측" + 0.007*"간부" + 0.007*"수사"')
(3, '0.008*"영상" + 0.007*"택시_기사" + 0.006*"차관" + 0.006*"손실_보상법" + 0.005*"이상섭_기자"')
(4, '0.015*"윤_전" + 0.011*"국민_힘" + 0.009*"사과" + 0.009*"총장" + 0.009*"장관"')
(5, '0.006*"국민" + 0.006*"여성" + 0.005*"박_의원" + 0.005*"국회" + 0.004*"모병제"')
(6, '0.011*"후보" + 0.008*"당_대표" + 0.006*"국민_힘" + 0.005*"생각" + 0.005*"박근혜_대통령"')
(7, '0.008*"사업" + 0.008*"얘기" + 0.007*"사업비" + 0.006*"초선_의원" + 0.005*"가상_자산"')
(8, '0.012*"군" + 0.009*"사건" + 0.009*"문제" + 0.007*"피해자" + 0.006*"부분"')
(9, '0.010*"중사" + 0.007*"사건" + 0.006*"공군" + 0.005*"장_중사" + 0.005*"수사"')
(10, '0.008*"시장" + 0.007*"아시아_경제" + 0.006*"시민" + 0.006*"사업" + 0.006*"평가"')
(11, '0.017*"국민_힘" + 0.015*"윤_전" + 0.009*"총장" + 0.009*"후보" + 0.007*"정치"')
(12, '0.019*"송_대표" + 0.012*"사과" + 0.011*"조_전" + 0.006*"문제" + 0.006*"장병"')
(13, '0.014*"배수찬" + 0.

In [19]:

def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob


In [20]:
lda_model = LdaModel(corpus, id2word=id2word, num_topics=NUM_TOPICS)

In [21]:
print(lda_model.alpha.shape) # (n_topics,)
print(lda_model.alpha.sum()) # 1.0

topic_term_prob = get_topic_term_prob(lda_model)
print(topic_term_prob.shape)     # (n_topics, n_terms)
print(topic_term_prob[0].sum())  # 1.0

(17,)
1.0
(17, 10259)
1.0


In [22]:
#lda_model.top_topics(corpus)

In [23]:
import pyLDAvis.gensim_models as gensimvis

prepared_data = gensimvis.prepare(lda_model, corpus, id2word)

In [24]:
import pyLDAvis

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
pyLDAvis.display(prepared_data)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
for i, topic_list in enumerate(ldamodel[corpus]):
    print(id_news[i],'번째 문서의 topic 비율은',topic_list)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


6 번째 문서의 topic 비율은 [(0, 0.019608187), (1, 0.019608187), (2, 0.01960819), (3, 0.019608187), (4, 0.019608198), (5, 0.019608188), (6, 0.01960819), (7, 0.019608187), (8, 0.019608187), (9, 0.019608187), (10, 0.019608187), (11, 0.686269), (12, 0.019608187), (13, 0.019608187), (14, 0.019608187), (15, 0.019608187), (16, 0.019608192)]
3444 번째 문서의 topic 비율은 [(0, 0.05882353), (1, 0.05882353), (2, 0.05882353), (3, 0.05882353), (4, 0.05882353), (5, 0.05882353), (6, 0.05882353), (7, 0.05882353), (8, 0.05882353), (9, 0.05882353), (10, 0.05882353), (11, 0.05882353), (12, 0.05882353), (13, 0.05882353), (14, 0.05882353), (15, 0.05882353), (16, 0.05882353)]
3448 번째 문서의 topic 비율은 [(0, 0.05882353), (1, 0.05882353), (2, 0.05882353), (3, 0.05882353), (4, 0.05882353), (5, 0.05882353), (6, 0.05882353), (7, 0.05882353), (8, 0.05882353), (9, 0.05882353), (10, 0.05882353), (11, 0.05882353), (12, 0.05882353), (13, 0.05882353), (14, 0.05882353), (15, 0.05882353), (16, 0.05882353)]
4925 번째 문서의 topic 비율은 [(2, 0.83227

5579 번째 문서의 topic 비율은 [(3, 0.23453693), (5, 0.40119523), (7, 0.35816288)]
5582 번째 문서의 topic 비율은 [(9, 0.8437183), (10, 0.1480289)]
5585 번째 문서의 topic 비율은 [(8, 0.13551638), (16, 0.855741)]
5588 번째 문서의 topic 비율은 [(16, 0.9890529)]
5591 번째 문서의 topic 비율은 [(6, 0.9927009)]
5594 번째 문서의 topic 비율은 [(2, 0.99350655)]
5597 번째 문서의 topic 비율은 [(4, 0.45861346), (6, 0.1395718), (16, 0.39659873)]
5600 번째 문서의 topic 비율은 [(6, 0.15702146), (7, 0.0689772), (9, 0.7659205)]
5603 번째 문서의 topic 비율은 [(12, 0.049346577), (14, 0.9450642)]
5606 번째 문서의 topic 비율은 [(8, 0.73406154), (9, 0.26097983)]
5609 번째 문서의 topic 비율은 [(3, 0.11435996), (5, 0.03075964), (16, 0.8313047)]
5612 번째 문서의 topic 비율은 [(4, 0.99372387)]
5615 번째 문서의 topic 비율은 [(11, 0.21684586), (14, 0.06304488), (15, 0.7055597)]
5618 번째 문서의 topic 비율은 [(5, 0.9923472)]
5621 번째 문서의 topic 비율은 [(12, 0.82403), (14, 0.17258845)]
5624 번째 문서의 topic 비율은 [(1, 0.99568224)]
5627 번째 문서의 topic 비율은 [(7, 0.99404216)]
5630 번째 문서의 topic 비율은 [(1, 0.15432775), (9, 0.6080489), (10, 0.23190

6041 번째 문서의 topic 비율은 [(4, 0.5861892), (11, 0.4101014)]
6044 번째 문서의 topic 비율은 [(2, 0.45017278), (11, 0.42738214), (14, 0.11914877)]
6047 번째 문서의 topic 비율은 [(9, 0.9968411)]
6050 번째 문서의 topic 비율은 [(4, 0.024336558), (12, 0.9696169)]
6053 번째 문서의 topic 비율은 [(4, 0.14388278), (6, 0.85131925)]
6056 번째 문서의 topic 비율은 [(15, 0.99384767)]
6059 번째 문서의 topic 비율은 [(12, 0.99436325)]
6062 번째 문서의 topic 비율은 [(0, 0.7554465), (5, 0.23878413)]
6065 번째 문서의 topic 비율은 [(4, 0.05371633), (11, 0.9424294)]
6068 번째 문서의 topic 비율은 [(1, 0.07150936), (9, 0.9240988)]
6071 번째 문서의 topic 비율은 [(9, 0.9963931)]
6074 번째 문서의 topic 비율은 [(8, 0.05707213), (9, 0.84074265), (12, 0.05297127), (14, 0.04601063)]
6077 번째 문서의 topic 비율은 [(1, 0.03344726), (4, 0.37616342), (11, 0.58077097)]
6080 번째 문서의 topic 비율은 [(4, 0.9822374)]
6083 번째 문서의 topic 비율은 [(5, 0.17921795), (8, 0.23539738), (11, 0.5808561)]
6086 번째 문서의 topic 비율은 [(4, 0.15317245), (6, 0.8423001)]
6089 번째 문서의 topic 비율은 [(0, 0.23159721), (6, 0.18563211), (8, 0.21728884), (11, 0.057553

In [27]:
pp = [[]]*(NUM_TOPICS+1)#NO Topic0
for i in range(NUM_TOPICS):
    pp[i] = prepared_data.sorted_terms(topic = i,_lambda=0.6).iloc[:20]

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [28]:
#index만 추출하기(index가 corpus의 index랑 동일함 ㅇㅇ)
idx_topic = [[]]*(NUM_TOPICS+1)
for i in range(1,len(pp)-1):
    idx_topic[i] = pp[i].iloc[:20].index

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:

#topic별 clustering을 위한 topic_docs 초기화(list[[]]*n 이런 식으로 하면 안됨)
topic_docs = []
topic_docs_save = []
for i in range(NUM_TOPICS+1):
    topic_docs.append([])
    topic_docs_save.append([])

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [30]:
#append doc_index in the topic_docs cluster
for i, topic_list in enumerate(ldamodel[corpus]):
    topic_list.sort(reverse=True,key = lambda element:element[1])
    n = topic_list[0][0] + 1
    topic_docs[n].append([i,topic_list[0][1]])


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
#append doc_index in the topic_docs cluster
for i, topic_list in enumerate(ldamodel[corpus]):
    topic_list.sort(reverse=True,key = lambda element:element[1])
    n = topic_list[0][0] + 1
    topic_docs_save[n].append([id_news[i],topic_list[0][1]])


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#topic_docs_save[2]
#반환해줄 실제 news index와 가중치 저장해둔 list

In [32]:
for i in topic_docs:
    i.sort(reverse=True,key = lambda element:element[1])


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
topic_cluster = []
for i in range(NUM_TOPICS+1):
    topic_cluster.append("")
for i in range(1,NUM_TOPICS+1):
    for j in topic_docs[i]:
        topic_cluster[i] = topic_cluster[i]+news[j[0]]
    

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
for i, topic_list in enumerate(ldamodel[corpus]):
    topic_list.sort(reverse=True,key= lambda element:element[1])
    print(i,'번째 문서의 topic 비율은',topic_list)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0 번째 문서의 topic 비율은 [(4, 0.68626916), (11, 0.019608246), (16, 0.01960818), (6, 0.019608177), (2, 0.019608175), (5, 0.019608175), (8, 0.019608174), (9, 0.019608174), (12, 0.019608174), (15, 0.019608174), (0, 0.019608172), (1, 0.019608172), (3, 0.019608172), (7, 0.019608172), (10, 0.019608172), (13, 0.019608172), (14, 0.019608172)]
1 번째 문서의 topic 비율은 [(0, 0.05882353), (1, 0.05882353), (2, 0.05882353), (3, 0.05882353), (4, 0.05882353), (5, 0.05882353), (6, 0.05882353), (7, 0.05882353), (8, 0.05882353), (9, 0.05882353), (10, 0.05882353), (11, 0.05882353), (12, 0.05882353), (13, 0.05882353), (14, 0.05882353), (15, 0.05882353), (16, 0.05882353)]
2 번째 문서의 topic 비율은 [(0, 0.05882353), (1, 0.05882353), (2, 0.05882353), (3, 0.05882353), (4, 0.05882353), (5, 0.05882353), (6, 0.05882353), (7, 0.05882353), (8, 0.05882353), (9, 0.05882353), (10, 0.05882353), (11, 0.05882353), (12, 0.05882353), (13, 0.05882353), (14, 0.05882353), (15, 0.05882353), (16, 0.05882353)]
3 번째 문서의 topic 비율은 [(2, 0.8322776), (

235 번째 문서의 topic 비율은 [(7, 0.99404216)]
236 번째 문서의 topic 비율은 [(9, 0.6080523), (10, 0.23190272), (1, 0.15432416)]
237 번째 문서의 topic 비율은 [(11, 0.5841353), (8, 0.4115589)]
238 번째 문서의 topic 비율은 [(9, 0.83911204), (1, 0.15471615)]
239 번째 문서의 topic 비율은 [(9, 0.7857438), (8, 0.15540102)]
240 번째 문서의 topic 비율은 [(2, 0.9828418), (16, 0.015669057)]
241 번째 문서의 topic 비율은 [(15, 0.9979355)]
242 번째 문서의 topic 비율은 [(9, 0.64359355), (10, 0.19618396), (1, 0.122479275), (0, 0.033127338)]
243 번째 문서의 topic 비율은 [(4, 0.99094486)]
244 번째 문서의 topic 비율은 [(7, 0.9952457)]
245 번째 문서의 topic 비율은 [(15, 0.9920887)]
246 번째 문서의 topic 비율은 [(1, 0.99568224)]
247 번째 문서의 topic 비율은 [(1, 0.99611014)]
248 번째 문서의 topic 비율은 [(5, 0.5244613), (3, 0.20224728), (11, 0.1551592), (14, 0.07994008), (9, 0.03448919)]
249 번째 문서의 topic 비율은 [(4, 0.99384713)]
250 번째 문서의 topic 비율은 [(9, 0.6316142), (11, 0.3133377), (4, 0.049202345)]
251 번째 문서의 topic 비율은 [(5, 0.958411), (14, 0.037107963)]
252 번째 문서의 topic 비율은 [(6, 0.8397643), (10, 0.14919877)]
253 번째 문

505 번째 문서의 topic 비율은 [(9, 0.99307716)]
506 번째 문서의 topic 비율은 [(6, 0.43032476), (11, 0.40690827), (16, 0.14523251)]
507 번째 문서의 topic 비율은 [(2, 0.99327624)]
508 번째 문서의 topic 비율은 [(16, 0.81640404), (2, 0.17269635)]
509 번째 문서의 topic 비율은 [(11, 0.8612916), (7, 0.1355445)]
510 번째 문서의 topic 비율은 [(10, 0.9855181)]
511 번째 문서의 topic 비율은 [(1, 0.96679187), (14, 0.02022903)]
512 번째 문서의 topic 비율은 [(13, 0.7113823), (12, 0.28306744)]
513 번째 문서의 topic 비율은 [(16, 0.9930253)]
514 번째 문서의 topic 비율은 [(10, 0.8411228), (15, 0.14811257)]
515 번째 문서의 topic 비율은 [(15, 0.63616174), (8, 0.3508583)]
516 번째 문서의 topic 비율은 [(6, 0.99847895)]
517 번째 문서의 topic 비율은 [(4, 0.99867016)]
518 번째 문서의 topic 비율은 [(9, 0.74572194), (2, 0.24764237)]
519 번째 문서의 topic 비율은 [(14, 0.54462224), (5, 0.3160367), (12, 0.12805326)]
520 번째 문서의 topic 비율은 [(9, 0.6887646), (14, 0.17997512), (1, 0.11817855)]
521 번째 문서의 topic 비율은 [(6, 0.40116027), (3, 0.18321474), (13, 0.15319712), (4, 0.14791323), (14, 0.099161655)]
522 번째 문서의 topic 비율은 [(12, 0.4695161), 

In [35]:
topic_cluster_sentences = []
tcs = []
for i in range(NUM_TOPICS+1):
    tcs.append([])
    topic_cluster_sentences.append([])

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [36]:
for i in range(1,NUM_TOPICS+1):
    mecab = Mecab()
    stop_words = stop.split(',')
    topic_cluster_sentences[i] = kss.split_sentences(topic_cluster[i])
    for j in topic_cluster_sentences[i]:
        sen_word = []
        sentences = mecab.nouns(j)
        for word in sentences :
            if word not in stop_words:
                sen_word.append(word)
        tcs[i].append(sen_word)

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
bigram_docs = []
for i in range(NUM_TOPICS+1):
    bigram_docs.append([])
#construct bigram_docs --> use the model made before
for i in range(1,len(tcs)):
    bigram_docs[i] = [bigram_model[nouns] for nouns in tcs[i]]

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
corpus_docs = []
for i in range(NUM_TOPICS+1):
    corpus_docs.append([])
for i in range(1,NUM_TOPICS+1):    
    corpus_docs[i] = [id2word.doc2bow(doc) for doc in bigram_docs[i]]

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
corp_doc_ref = []
for i in range(NUM_TOPICS+1):
    corp_doc_ref.append([])


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
stime = time.time()

for i in range(1,NUM_TOPICS+1):
    if len(corpus_docs[i]) is not 0:
        for j in range(len(corpus_docs[i])):
            corp_doc_ref[i].append([])
            for k in range(len(corpus_docs[i][j])):
                a = corpus_docs[i][j][k][0]
                corp_doc_ref[i][j].append(a)

print(time.time()-stime)

0.03825116157531738


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [41]:
#키워드 포함된 문장만 추출
stime = time.time()
corp_doc_topic = []
for i in range(NUM_TOPICS+1):
    corp_doc_topic.append([])
for i in range(1,len(corp_doc_ref)):
    for j in range(len(corp_doc_ref[i])):
        if len(set(idx_topic[i-1]).intersection(corp_doc_ref[i][j])) is not 0:
            corp_doc_topic[i].append(bigram_docs[i][j])

print(time.time()-stime)

0.02292180061340332


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


corp_doc_topic[N][M] : N개의 topic, M개의 문장 키워드 포함한 문장만 뽑아낸 것

In [42]:
def vectorize_sents(corp_doc_topic = corp_doc_topic, min_count=2, tokenizer="mecab", noun=True):
   

    vectorizer = CountVectorizer(tokenizer=lambda x: x,lowercase=False)

    vec = vectorizer.fit_transform(corp_doc_topic)
    vocab_idx = vectorizer.vocabulary_
    idx_vocab = {idx: vocab for vocab, idx in vocab_idx.items()}
    
    return vec, vocab_idx, idx_vocab

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [43]:
def word_similarity_matrix(x, min_sim=0.3):
    sim_mat = 1 - pairwise_distances(x.T, metric="cosine")
    sim_mat[np.where(sim_mat <= min_sim)] = 0

    return sim_mat

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [44]:
def word_graph(
    corp_doc_topic = corp_doc_topic,
    min_count=2,
    min_sim=0.3,
    tokenizer="mecab",
    noun=True,
):

    mat, vocab_idx, idx_vocab = vectorize_sents(
        corp_doc_topic,min_count=min_count, tokenizer=tokenizer, noun=noun
    )

    mat = word_similarity_matrix(mat, min_sim=min_sim)

    return mat, vocab_idx, idx_vocab

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
def pagerank(x: np.ndarray, df=0.85, max_iter=50, method="iterative"):

    assert 0 < df < 1

    A = normalize(mat, axis=0, norm="l1")
    N = np.ones(A.shape[0]) / A.shape[0]

    if method == "iterative":
        R = np.ones(A.shape[0])
        for _ in range(max_iter):
            R = df * np.matmul(A, R) + (1 - df) * N
    elif method == "algebraic":
        R = np.linalg.inv((I - df * A))
        R = np.matmul(R, (1 - df) * N)

    return R

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
keywords = []

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [47]:
for i in range(len(corp_doc_topic)):
    if len(corp_doc_topic[i])!= 0:
        mat,vocab_idx, idx_vocab = word_graph(corp_doc_topic[i])
        R = pagerank(mat,method ="iterative")
        topk = 10
        idxs = R.argsort()[-topk:]
        keywords.append([(idx, R[idx], idx_vocab[idx]) for idx in reversed(idxs)])
        #keyword = [(R[idx]) for idx in reversed(idxs)]


/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [48]:
keywords

/Users/heejaesuh/opt/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(55, 0.0039040245011184907, '관련'),
  (214, 0.00377633858145882, '반'),
  (151, 0.00377633858145882, '대응'),
  (525, 0.00377633858145882, '지적'),
  (380, 0.00377633858145882, '원인'),
  (524, 0.00377633858145882, '지원'),
  (204, 0.0036487971681422042, '민심'),
  (176, 0.0036487971681422042, '뜻'),
  (315, 0.003441597705566129, '소장'),
  (149, 0.0032449836197869053, '대선')],
 [(231, 0.0027607788057860882, '문'),
  (103, 0.0026865422213088255, '기구'),
  (538, 0.0026865422213088255, '윤준병'),
  (614, 0.0026865422213088255, '임호'),
  (789, 0.0026865422213088255, '최혜영'),
  (581, 0.0026865422213088255, '이장섭'),
  (249, 0.0026865422213088255, '박광온'),
  (694, 0.0026865422213088255, '조승래'),
  (109, 0.0026865422213088255, '기자'),
  (121, 0.0026865422213088255, '김민석')],
 [(325, 0.004148964211203993, '의원'),
  (198, 0.0037303113289000617, '사건'),
  (460, 0.0034471688361027257, '최종'),
  (212, 0.003361327309332382, '생각'),
  (58, 0.0032485740291397142, '관련'),
  (505, 0.0032259134256840006, '행정'),
  (490, 0.003200831996

In [ ]:
뿐,면,반,관련,기대,제기,우려,번,지적,배포,금지